In [16]:
import os
from skimage import io, filters, transform

def threshold_images(input_folder, output_folder, max_width, max_height, threshold_value):

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.png'):
            # Read the image
            img_path = os.path.join(input_folder, filename)
            image = io.imread(img_path, as_gray=True)

            # Resize the image
            # Calculate the scaling factor to fit within the specified range
            scale_factor = min(max_width / image.shape[1], max_height / image.shape[0])
            resized_image = transform.resize(image, (int(image.shape[0] * scale_factor), int(image.shape[1] * scale_factor)))

            # Sharpening (using unsharp masking)
            sharpened_image = filters.unsharp_mask(resized_image, radius=3, amount=3)
            
            # Apply thresholding
            thresholded_image = sharpened_image > threshold_value

            # Sharpening (using unsharp masking)
            sharpened_image = filters.unsharp_mask(thresholded_image, radius=1, amount=1)
            
            # Save thresholded image to output folder
            output_path = os.path.join(output_folder, filename)
            io.imsave(output_path, sharpened_image.astype('uint8') * 255)  # Convert boolean array to uint8


#input test
input_folder = 'C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/test'
output_folder = "C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/enhanced_images"

#actual folder
# input_folder = 'C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/mydispense_meds_proceessing'
# output_folder = 'C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/resized_images'
max_width = 600  # Specify your maximum width
max_height = 600  # Specify your maximum height
threshold_value=0.76 #0.76

# Apply try_all_threshold to PNG images in the input folder
threshold_images(input_folder, output_folder, max_width, max_height, threshold_value)